In [2]:
# Import packages
import cv2
import os
from matplotlib import pyplot as plt
%matplotlib tk
import pandas as pd
import numpy as np
from skimage import io, color

In [3]:
# Figure out directories
main = os.getcwd()
print(main)
MainImagePath = '/Users/maureenkeenan/Desktop/Kaggle/Kaggle-Cloud-Classification'
Train_Images  = '/data/train_images'

/Users/maureenkeenan/Desktop/Kaggle/Kaggle-Cloud-Classification/notebooks


In [93]:
# Look at the structure of the train.csv
os.chdir(MainImagePath)
img_info = pd.read_csv('./data/train.csv')
img_info.head()

,Image_Label,EncodedPixels
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...
2,0011165.jpg_Gravel,NaN
3,0011165.jpg_Sugar,NaN
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...


## Next Steps
* Drop rows with no encoded pixels (no cloud formation found in that image)
* Split the Image_Label into the jpg name and cloud label
* Plot first figure with both bounding boxes on it

In [95]:
# Remove rows with EncodedPixels == NaN
img_info = img_info.dropna() #needs to be set equal to update

# Split filename and classification
img_name = img_info.Image_Label.str.split(pat ="_").str[0]
cloud = img_info.Image_Label.str.split(pat ="_").str[1]

# Make new columns
img_info['name'] = img_name
img_info['cloud'] = cloud

# Look again at the df
img_info.head()

,Image_Label,EncodedPixels,name,cloud
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...,0011165.jpg,Fish
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...,0011165.jpg,Flower
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...,002be4f.jpg,Fish
5,002be4f.jpg_Flower,1339279 519 1340679 519 1342079 519 1343479 51...,002be4f.jpg,Flower
7,002be4f.jpg_Sugar,67495 350 68895 350 70295 350 71695 350 73095 ...,002be4f.jpg,Sugar


In [21]:
# Load the first image using cv2
img = cv2.imread(os.path.join('./data/train_images',img_info.name[0]))
type(img)

numpy.ndarray

In [96]:
# Plot the first look 
fig = plt.figure()
plt.title(img_info['cloud'][0])
plt.imshow(img)
plt.show()

**Comments**
* That wasn't super helpful as this image has 2 cloud regions
* Use the EncodedPixels to determine which part of the image is which and plot again

In [89]:
# Get decoder
h,w,ch = img.shape #no '()'
num_pixels = h * w

# Get list of locations in image and convert from string to int and reshape
temp = img_info['EncodedPixels'][0].split()
temp = np.array([int(s) for s in temp])
regions = temp.reshape((int(len(temp)/2),2))

(998, 2)

In [97]:
# Get x,y coordinates out of the regions info
mask = np.zeros(h*w)
for r in range(len(regions)):
    start = regions[r][0] 
    end  = regions[r][1] 
    mask[start:start+end] = 1
mask = mask.reshape((h,w), order = 'F')
plt.plot(mask)
plt.show()

In [104]:
import matplotlib.patches as patches

def getPatch(mask):
    # Get mask coordinates
    coords = np.column_stack(np.where(mask > 0))
    
    # Extract corners and width
    y0 = coords[:,0].max()
    y1 = coords[:,0].min()

    x0 = coords[:,1].min()
    x1 = coords[:,1].max()
    
    # Make patch
    xy = (x0,y0)
    width = x1 - x0
    height = y1 - y0
    if img_info['cloud'][0] == 'Fish':
        rect = patches.Rectangle(xy, width, height,linewidth=1,edgecolor='r',
                             facecolor='none')
    elif img_info['cloud'][0] == 'Flower':
        rect = patches.Rectangle(xy, width, height,linewidth=1,edgecolor='y',
                             facecolor='none')
    
    return rect

In [105]:
# Create figure and axes
fig,ax = plt.subplots(1)

# Display the image
ax.imshow(img)

# Make patch
rect = getPatch(mask)

# Add the patch to the Axes
ax.add_patch(rect)

plt.show()